This notebook regroups the code sample of the video below, which is a part of the [Hugging Face course](https://huggingface.co/course).

In [1]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/A5IWIxsHLUw?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Install the Transformers and Datasets libraries to run this notebook.

In [2]:
! pip install datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


You will need an authentication token with your Hugging Face credentials to use the `push_to_hub` method. Execute `huggingface-cli login` in your terminal or by uncommenting the following cell:

In [3]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.0 MB/s eta 0:00:00


In [4]:
! pip install accelerate -U

In [5]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [6]:
import numpy as np

from datasets import load_dataset, load_metric
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

In [7]:
checkpoint = "bert-base-cased"

In [8]:
raw_datasets = load_dataset("glue", "mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [9]:
i = 0
raw_datasets['train']['sentence1'][i], raw_datasets['train']['sentence2'][i], raw_datasets['train']['label'][0]

('Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 1)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments(
    "finetuned-bert-mrpc",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    log_level="error",
    push_to_hub=True,
    push_to_hub_model_id="finetuned-bert-mrpc",
    # push_to_hub_organization="huggingface",
    # push_to_hub_token="my_token",
)

data_collator = DataCollatorWithPadding(tokenizer)

metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1716: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case eagle0504/finetuned-bert-mrpc).
  warnings.warn(
<ipython-input-10-1487c5462d68>:27: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom

In [11]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.545700,0.434578,0.801471,0.866116
2,0.309400,0.405065,0.852941,0.897959
3,0.142400,0.447352,0.850490,0.895726


TrainOutput(global_step=690, training_loss=0.33250519019969993, metrics={'train_runtime': 62.7517, 'train_samples_per_second': 175.358, 'train_steps_per_second': 10.996, 'total_flos': 446520016497120.0, 'train_loss': 0.33250519019969993, 'epoch': 3.0})

## Push to hub from the Trainer directly

The `Trainer` has a new method to directly upload the model, tokenizer and model configuration in a repo on the [Hub](https://huggingface.co/). It will even auto-generate a model card draft using the hyperparameters and evaluation results!

In [13]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

events.out.tfevents.1706651875.1b5ada8287fe.2014.0:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/eagle0504/finetuned-bert-mrpc/commit/191c365badcb16b10732d85914cf38cb6e1eaf77', commit_message='End of training', commit_description='', oid='191c365badcb16b10732d85914cf38cb6e1eaf77', pr_url=None, pr_revision=None, pr_num=None)

If you are using your own training loop, you can push the model and tokenizer separately (and you will have to write the model card yourself):

In [ ]:
# model.push_to_hub("finetuned-bert-mrpc")
# tokenizer.push_to_hub("finetuned-bert-mrpc")

## You can load your model from anywhere using from_pretrained!

In [14]:
from transformers import AutoModelForSequenceClassification

model_name = "eagle0504/finetuned-bert-mrpc"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

## You can use your model in a pipeline!

In [15]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model_name)

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [16]:
classifier("My name is Sylvain. [SEP] My name is Lysandre")

[{'label': 'LABEL_0', 'score': 0.820957601070404}]

## Updating a problematic file is super easy!

In [17]:
model.config.label2id = {"not equivalent": 0, "equivalent": 1}

In [18]:
model.config.id2label = {0: "not equivalent", 1: "equivalent"}

In [19]:
model.config.push_to_hub("finetuned-bert-mrpc")

CommitInfo(commit_url='https://huggingface.co/eagle0504/finetuned-bert-mrpc/commit/7606159bf42f34ce1b4ab96f68b8664f295327cd', commit_message='Upload config', commit_description='', oid='7606159bf42f34ce1b4ab96f68b8664f295327cd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
classifier = pipeline("text-classification", model=model_name)

classifier("My name is Sylvain. [SEP] My name is Lysandre")

[{'label': 'not equivalent', 'score': 0.7789641618728638}]